In [1]:
import nltk
nltk.data.find('tokenizers/punkt')
print("✅ El recurso 'punkt' está disponible.")

✅ El recurso 'punkt' está disponible.


In [3]:
import nltk
nltk.data.path.append('/Users/pavt/nltk_data')  # Asegurar que usa la ruta correcta
